### Create mosaic

In [ ]:
# If needed:
# !pip install rasterio tqdm

from pathlib import Path
import numpy as np
import rasterio
from rasterio.merge import merge
from rasterio.vrt import WarpedVRT
from rasterio.windows import Window
from rasterio.enums import Resampling
from tqdm import tqdm

# --- INPUTS ---
INPUT_DIR = r"C:\temp\timor_leste\greening"
OUTPUT_TIF = r"C:\temp\timor_leste\greening\TimorLeste_GREENER_2019_2025_20m_mosaic.tif"

# Only pick your exported tiles (change the glob if you want all .tif)
tif_paths = sorted([str(p) for p in Path(INPUT_DIR).glob("*_NDVI_EVI_GREEN_INCREASE_2019_2025_20m*.tif")])
if not tif_paths:
    raise FileNotFoundError("No matching GeoTIFFs found in: " + INPUT_DIR)

print(f"Found {len(tif_paths)} GeoTIFF(s).")

# --- READ REFERENCE DATASET ---
with rasterio.open(tif_paths[0]) as ref:
    ref_profile = ref.profile.copy()
    ref_crs     = ref.crs
    ref_res     = ref.res
    ref_dtype   = ref.dtypes[0]
    ref_count   = ref.count
    ref_nodata  = ref_profile.get("nodata")
    if ref_nodata is None:
        # Choose a sensible default
        ref_nodata = -9999.0 if np.issubdtype(np.dtype(ref_dtype), np.floating) else 0
    print(f"Reference bands={ref_count}, dtype={ref_dtype}, crs={ref_crs}, res={ref_res}, nodata={ref_nodata}")

# --- OPEN ALL DATASETS (wrap in VRT if CRS/res mismatch) ---
datasets = []
try:
    for fp in tqdm(tif_paths, desc="Opening rasters"):
        src = rasterio.open(fp)
        # Sanity checks
        if src.count != ref_count:
            src.close()
            raise ValueError(f"Band count mismatch in {fp}: {src.count} vs {ref_count} (reference)")

        # Use on-the-fly WarpedVRT if CRS or resolution differs
        if (src.crs != ref_crs) or (tuple(map(float, src.res)) != tuple(map(float, ref_res))):
            vrt = WarpedVRT(
                src,
                crs=ref_crs,
                res=ref_res,
                nodata=ref_nodata,
                dtype=ref_dtype,
                resampling=Resampling.nearest  # safe for categorical/mask-like bands
            )
            datasets.append(vrt)
        else:
            datasets.append(src)

    # --- MERGE (mosaic) ---
    # method='first' keeps the first valid pixel encountered; good when tiles mostly don't overlap.
    # If you want "maximum" across overlaps (e.g., for greening masks), use method='max'.
    print("Merging… (this may take a while)")
    mosaic, out_transform = merge(
        datasets,
        nodata=ref_nodata,
        dtype=ref_dtype,
        method="first"   # or "max" if you prefer max across overlaps
    )
    out_count, out_height, out_width = mosaic.shape
    print(f"Mosaic shape: bands={out_count}, height={out_height}, width={out_width}")

    # --- WRITE OUTPUT (tiled, compressed, BigTIFF if needed) ---
    out_profile = ref_profile.copy()
    out_profile.update({
        "height": out_height,
        "width": out_width,
        "transform": out_transform,
        "count": out_count,
        "nodata": ref_nodata,
        "compress": "LZW",
        "tiled": True,
        "blockxsize": 512,
        "blockysize": 512,
        "BIGTIFF": "IF_SAFER"  # auto-upgrade to BigTIFF if > 4 GB
    })

    # Write in row-tiles with a progress bar (friendlier on memory for very large rasters)
    tile_h = 1024
    with rasterio.open(OUTPUT_TIF, "w", **out_profile) as dst:
        H, W = out_height, out_width
        for row in tqdm(range(0, H, tile_h), desc="Writing tiles"):
            h = min(tile_h, H - row)
            window = Window(col_off=0, row_off=row, width=W, height=h)
            dst.write(mosaic[:, row:row+h, :], window=window)

        # Optional: build overviews for faster display in GIS
        try:
            factors = []
            f = 2
            while min(H//f, W//f) >= 256:
                factors.append(f); f *= 2
            if factors:
                dst.build_overviews(factors, Resampling.nearest)
                dst.update_tags(ns="rio_overview", resampling="nearest")
                print(f"Built overviews: {factors}")
        except Exception as e:
            print("Overview build skipped:", e)

    print("✅ Done:", OUTPUT_TIF)

finally:
    # Clean up VRTs and datasets
    for ds in datasets:
        try:
            ds.close()
        except Exception:
            pass


### Raster calculator

In [ ]:
# !pip install rasterio numpy tqdm

import numpy as np
import rasterio
from tqdm import tqdm

in_path  = r"C:\temp\timor_leste\TimorLeste_GREENER_2019_2025_20m_mosaic.tif"
out_path = r"C:\temp\timor_leste\TimorLeste_GREENER_2019_2025_20m_max_27_30.tif"

band_indexes = [27, 28, 29, 30]

with rasterio.open(in_path) as src:
    masked_bands = []
    
    # Show progress as each band is read
    for i in tqdm(band_indexes, desc="Reading bands"):
        masked_bands.append(src.read(i, masked=True))

    # Stack and compute pixelwise maximum (with progress feedback)
    print("Computing maximum across bands…")
    stack = np.ma.stack(masked_bands, axis=0)
    max_arr = stack.max(axis=0)  # masked array

    # Handle nodata
    nodata = src.nodata
    if nodata is None:
        nodata = -9999 if np.issubdtype(np.dtype(src.dtypes[0]), np.integer) else np.float32(np.nan)

    filled = max_arr.filled(nodata).astype(src.dtypes[0])

    profile = src.profile.copy()
    profile.update(count=1, nodata=nodata, compress="deflate")

    # Write out with a progress bar (simulated for 1 band)
    with rasterio.open(out_path, "w", **profile) as dst:
        for j in tqdm(range(1, 2), desc="Writing output"):
            dst.write(filled, j)

print("✅ Done:", out_path)
